<a href="https://colab.research.google.com/github/SwethaMagesh/sankshepika-mlpro/blob/main/Abstractive_Summarizer_using_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle

Uncomment if running on colab (gpu has the dataset)

In [2]:
!wget https://zenodo.org/records/7152317/files/dataset.zip?download=1
!unzip '/content/dataset.zip?download=1'

Streaming output truncated to the last 5000 lines.
  inflating: dataset/IN-Abs/train-data/summary/4620.txt  
  inflating: dataset/IN-Abs/train-data/summary/463.txt  
  inflating: dataset/IN-Abs/train-data/summary/4630.txt  
  inflating: dataset/IN-Abs/train-data/summary/4631.txt  
  inflating: dataset/IN-Abs/train-data/summary/4643.txt  
  inflating: dataset/IN-Abs/train-data/summary/4647.txt  
  inflating: dataset/IN-Abs/train-data/summary/4652.txt  
  inflating: dataset/IN-Abs/train-data/summary/4667.txt  
  inflating: dataset/IN-Abs/train-data/summary/4668.txt  
  inflating: dataset/IN-Abs/train-data/summary/4682.txt  
  inflating: dataset/IN-Abs/train-data/summary/4690.txt  
  inflating: dataset/IN-Abs/train-data/summary/4693.txt  
  inflating: dataset/IN-Abs/train-data/summary/4697.txt  
  inflating: dataset/IN-Abs/train-data/summary/4711.txt  
  inflating: dataset/IN-Abs/train-data/summary/4712.txt  
  inflating: dataset/IN-Abs/train-data/summary/4715.txt  
  inflating: dataset/I

In [3]:
# !unzip '/content/dataset.zip?download=1'

### Loading Data

In [4]:
import os
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
def prepare_val_list():
    val_files = []  # Add the validation files to be used
    for i in range(1, 50):
        x = random.randint(1, 7028)
        if x not in val_files:
            val_files.append(x)

    val_files = list(map(lambda x: str(x) + '.txt', val_files))
    return val_files

def getData(dataPath,  val=0):

    documentPath = f'{dataPath}/judgement'
    summaryPath = f'{dataPath}/summary'
    dataset = {'document': [], 'summary': []}
    count = 0
    for file in os.listdir(documentPath):
        count += 1
        if os.stat(f'{documentPath}/{file}').st_size == 0 or os.stat(f'{summaryPath}/{file}').st_size == 0:
            continue
        doc_in = open(f'{documentPath}/{file}', 'r', encoding='utf8')
        doc_lines = [line.strip() for line in doc_in.readlines()]
        summ_in = open(f'{summaryPath}/{file}', 'r', encoding='utf8')
        summ_lines = [line.strip() for line in summ_in.readlines()]
        if len(doc_lines) == 0 or len(summ_lines) == 0:
            continue

        if val == 0 and file not in val_files:
            dataset['document'].append(' '.join(doc_lines))
            dataset['summary'].append(' '.join(summ_lines))
        if val == 1 and file in val_files:
            dataset['document'].append(' '.join(doc_lines))
            dataset['summary'].append(' '.join(summ_lines))

    df = pd.DataFrame(dataset)
    return df


dataPath = "/content/dataset/IN-Abs" #
# dataPath = "./dataset/IN-Abs"

val_files = prepare_val_list()
train_df = getData( f'{dataPath}/train-data')
val_df = getData(f'{dataPath}/train-data')

In [5]:
val_df.head()

,document,summary
0,ition (Crl.) No. 648 of 1987 etc. (Under Artic...,The petitioners who have been convicted for th...
1,Civil Appeal No. 1384 of 1935 From the Judgmen...,G.O.Ms. No. 559 dated May 4. 1961 issued by th...
2,Appeal No.903 of 1978. From the Judgment and O...,"The Union of India, the appellant, approached ..."
3,ivil Appeal No. 3071 of 1988. ? 199 From the J...,"The respondent, a bank employee, was chargeshe..."
4,Civil Appeal No. 4 (NCE) of 1987. From the Jud...,% The first respondent filed an election petit...


In [6]:
val_df.shape

(6983, 2)

In [14]:
document = val_df['document']
summary = val_df['summary']

In [15]:
document[30], summary[30]

('ION: Criminal Appeal No. 120 of 1961. Appeal by special leave from the judgment and order dated March 17, 1961 of the Punjab High Court in Criminal Writ No. 2 of 1961. WITH Petition No. 147 of 1961. Petition under article 32 of the Constitution of India for enforcement of Fundamental Rights. The appellant/petitioner in person. H. section Doabia, Additional Advocate General, Punjab, Gopal Singh and P. D. Menon. for respondent (in the appeal and the petition.) 1961. November 2. \'The Judgment of Sinha, C. J., Subba Rao, Shah and Mudholkar, JJ, was delivered by Subba Rao, J. Dayal, J. delivered a separate Judgment. SUBBA RAO, J. Both these matters are connected and raise the same questions, and they may be disposed of together. Ranbir Singh Sehgal, the petitioner in the writ petition, is now a prisoner in the Central Jail Ambala, in the State of Punjab. He was prosecuted for committing offence in different places. On June 13, 1961, he was convicted by the Additional District Magistrate,

### Preprocessing

In [12]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
def sentence_tokenizer(doc):
  sentences = sent_tokenize(doc)
  result = '<eos> '.join(sentences)
  return result


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
# for decoder sequence
summary = summary.apply(lambda x: '<go> ' + x + ' <stop>')
summary = summary.apply(sentence_tokenizer)
summary.head()

0    <go> The petitioners who have been convicted f...
1    <go> G.O.Ms.<eos> No.<eos> 559 dated May 4.<eo...
2    <go> The Union of India, the appellant, approa...
3    <go> The respondent, a bank employee, was char...
4    <go> % The first respondent filed an election ...
Name: summary, dtype: object

#### Tokenizing the texts into integer tokens

In [17]:
# since < and > from default tokens cannot be removed
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = '<unk>'

In [18]:
document_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token)

In [19]:
summary_tokenizer.fit_on_texts(["<go>","<stop>","<eos>"])

In [20]:
document_tokenizer.fit_on_texts(document)
summary_tokenizer.fit_on_texts(summary)

In [21]:
inputs = document_tokenizer.texts_to_sequences(document)
targets = summary_tokenizer.texts_to_sequences(summary)

In [22]:
summary.iloc[2]

"<go> The Union of India, the appellant, approached the High Court under Article 226 of the Constitution, challenging notices of demand issued by the Municipal Council, respond ent No.1, claiming tax to the tune of Rs.<eos> 28,400 by way of 'Service charges ' due for the period 1954 to 1960, under Article 285 of the Constitution read with Section 135 of the Indian Railways Act, 1890.<eos> The High Court dismissed the writ petition negativing the claim, against which, this appeal by special leave was filed.<eos> Allowing the appeal, this Court, HELD 1.<eos> Section 135 of the Indian Railways Act, 1890 gets saved under Article 285(1) of the Constitution itself.<eos> The said Article provides that property of the Union shall, save in so far as Parliament may by law otherwise provide, be exempt from all taxes imposed by the State or by any authority within a State.<eos> [185 A B] 2.<eos> Section 1315 of the Railways Act permits taxation of Railways by the local authority in the manner give

In [23]:
summary_tokenizer.texts_to_sequences(["This is a test"])

[[37, 13, 8, 680]]

In [24]:
summary_tokenizer.sequences_to_texts([[14, 228, 172, 761]])

['not certain rs term']

In [25]:
encoder_vocab_size = len(document_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(130045, 66923)

#### Obtaining insights on lengths for defining maxlen

In [26]:
document_lengths = pd.Series([len(x) for x in document])
summary_lengths = pd.Series([len(x) for x in summary])

In [27]:
document_lengths.describe()

count      6983.000000
mean      25025.905198
std       29500.627327
min         592.000000
25%       11536.500000
50%       17942.000000
75%       28600.500000
max      808118.000000
dtype: float64

In [28]:
summary_lengths.describe()

count      6983.000000
mean       5006.927538
std        5363.819732
min         169.000000
25%        2518.000000
50%        3802.000000
75%        5753.000000
max      163811.000000
dtype: float64

In [29]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
encoder_maxlen = 400
decoder_maxlen = 75

### Padding/Truncating sequences for identical sequence lengths

In [30]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

### Creating dataset pipeline

In [31]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [32]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [33]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional

In [34]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [35]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


### Masking

- Padding mask for masking "pad" sequences
- Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [36]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [37]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model

#### Scaled Dot Product

In [38]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

#### Multi-Headed Attention

In [39]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)

        return output, attention_weights

### Feed Forward Network

In [40]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

#### Fundamental Unit of Transformer encoder

In [41]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


#### Fundamental Unit of Transformer decoder

In [42]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)


    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


#### Encoder consisting of multiple EncoderLayer(s)

In [43]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x


#### Decoder consisting of multiple DecoderLayer(s)

In [44]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        return x, attention_weights


#### Finally, the Transformer

In [45]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


### Training

In [46]:
# hyper-params
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
EPOCHS = 20

#### Adam optimizer with custom learning rate scheduling

#### Defining losses and other metrics

In [47]:

optimizer = tf.keras.optimizers.Adam(0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [48]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [49]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [50]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

#### Transformer

In [51]:
transformer = Transformer(
    num_layers,
    d_model,
    num_heads,
    dff,
    encoder_vocab_size,
    decoder_vocab_size,
    pe_input=encoder_vocab_size,
    pe_target=decoder_vocab_size,
)

#### Masks

In [52]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


#### Checkpoints

In [53]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

#### Training steps

In [54]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp,
            True,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()

    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)

        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


### Inference

#### Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [56]:
def evaluate(input_document):
    input_document = document_tokenizer.texts_to_sequences([input_document])
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [summary_tokenizer.word_index["<go>"]]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input,
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index["<stop>"]:
            return tf.squeeze(output, axis=0), attention_weights
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [67]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # not printing <go> token
    temp = summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document
    sentences = temp.split('<eos>')
    return '. '.join([x.strip().capitalize() for x in sentences])

In [68]:
summarize(train_df.iloc[3001]['document'])

'The appellant was a manufacturer of iron and steel products. The respondent was a manufacturer of iron ore and steel products. The appellant was a manufacturer and steel products was carrying on business and selling agents. The appellant claimed exemption under section 3 1 of the u p. Sales tax act 1948. The respondent claimed that the sales tax officer had no right to purchase sales tax on the sales'